[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/02-langsmith.ipynb)

#### LangChain Essentials Course

# LangSmith Starter

LangSmith is a built-in observability service and platform that integrates _very easily_ with LangChain. You don't _need_ to use LangSmith for this course, but it can be very helpful in understanding what is happening, _and_ we recommend using it beyond this course for general development with LangChain — with all of that in mind we would recommend spending a little bit of time to get familiar with LangSmith.

In [2]:
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community \
  langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00


---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

## Setting up LangSmith

LangSmith does require an API key, but it comes with a generous free tier. You can sign up an account and get your API key [here](https://smith.langchain.com).

When using LangSmith, we need to setup our environment variables _and_ provide our API key, like so:

In [3]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-langsmith-openai"

Enter LangSmith API Key: ··········


In most cases, this is all we need to start seeing logs and traces in the [LangSmith UI](https://smith.langchain.com). By default, LangChain will trace LLM calls, chains, etc. We'll take a look at a quick example of this below.

## Default Tracing

As mentioned, LangSmith traces a lot of data without us needing to do anything. Let's see how that looks. We'll start by initializing our LLM. Again, this will need an [API key](https://platform.openai.com/api-keys).

In [7]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass(
    "Enter OpenAI API Key: "
)

llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

Enter OpenAI API Key: ··········


Let's invoke our LLM and then see what happens in the LangSmith UI.

In [8]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bgsx7RtHd8E059jmC2gbpAzKsK1Ef', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ea1e6ae4-5360-4ec4-846c-20a6bdf39ffa-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

After this we should see that a new project (`aurelioai-langchain-course-langsmith-openai`) has been created in the LangSmith UI. Inside that project, we should see the trace from our LLM call:


![LangSmith LLM Trace](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-01.jpg?raw=1)

By default, LangSmith will capture plenty — however, it won't capture functions from outside of LangChain. Let's see how we can trace those.

## Tracing Non-LangChain Code

LangSmith can trace functions that are not part of LangChain, we just need to add the `@traceable` decorator. Let's try this for a few simple functions.

In [9]:
from langsmith import traceable
import random
import time


@traceable
def generate_random_number():
    return random.randint(0, 100)

@traceable
def generate_string_delay(input_str: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_str} ({number})"

@traceable
def random_error():
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "No error"

Let's run these a few times and see what happens.

In [10]:
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("Hello")
    try:
        random_error()
    except ValueError:
        pass

  0%|          | 0/10 [00:00<?, ?it/s]

Those traces should now be visible in the LangSmith UI, again under the same project:

![LangSmith Traces](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-02.jpg?raw=1)

We can various metrics here for each run. First, ofcourse, the run name. We can see any inputs and outputs from each run, we can see if the run raised any errors, it's start time, and latency. Inside the UI we can also filter for specific runs, like so:

![LangSmith UI filters](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-03.jpg?raw=1)

There are various other things we can do within the UI, but we'll leave that for you to explore.

Finally, we can also modify our traceable names if we'd like to make them more readable inside the UI. For example:

In [11]:
from langsmith import traceable

@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "I'm great how are you?"

Let's run this a few times and see what we get in LangSmith.

In [12]:
for _ in tqdm(range(10)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

  0%|          | 0/10 [00:00<?, ?it/s]

Let's filter for the `Chitchat Maker` traceable and see our results.

![LangSmith Chitchat Maker runs](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-04.jpg?raw=1)

We can see our runs and their related metadata! That's it for this intro to LangSmith. As we work through the course we will (optionally) refer to LangSmith for digging into our runs.